In [ ]:
def problem4(A, B, D, alpha, riskAversion, maxGross, allowShort=False, 
             maxNet=None, minNet=None):
    n = alpha.size
    cov = A.T.dot(B).dot(A) + D
    rootCov = matrix(np.linalg.cholesky(cov).T)
    
    coeff = matrix([riskAversion, matrix(-1.0*alpha)])
    
    G0 = matrix([]) 
    h0 = matrix([])
    
    zn = matrix(0.0, (1,n))
    en = matrix(np.eye(n))
    znn = znn = matrix(0.0, (n,n))
    znn = matrix(0.0, (n,n))
    
    G1 = matrix([[-1, zn.T],[zn, -1.0*rootCov]])
    h1 = matrix(0.0, (n+1,1)) 
    
    if allowShort:
        #введем n новых переменных 
        coeff = matrix([coeff, matrix(0.0, (n,1))])
        z = matrix(0.0, (n+1,n))
        G1 = matrix([[G1], [z]])
        
        G0 = matrix([ [-1.0], [matrix(0.0, (1,2*n))] ]) #(-1, 0, ..., 0)
        add_G0 = matrix([[matrix(0.0, (1,n+1))], [matrix(1.0, (1,n))] ]) #(0,..,0,1,...1)
        G0 = matrix([G0,add_G0])
        add_G0 = matrix([[zn.T],[ matrix([[en],[-1.0*en]])]]) #[O, E, -E], size n*(2n+1)
        G0 = matrix([G0,add_G0])
        add_G0 = matrix([[zn.T],[ matrix([[-1.0*en],[-1.0*en]])]]) #[O, -E, -E], size n*(2n+1)
        G0 = matrix([G0,add_G0])
        add_G0 = matrix([ [matrix(0.0, (n, n+1))] ,[-1.0*en]])#[O, O, -E], size n*(2n+1)
        G0 = matrix([G0,add_G0])
        h0 = matrix([0.0, float(maxGross)])
        add_h0 = matrix([0 for i in range(3*n)])
        h0 = matrix([h0,add_h0])

        if maxNet is not None:
            add_G0 = matrix([ [zn.T], [en], [znn]])
            add_h0 = matrix( float(maxNet), (n,1))
            G0 = matrix([G0, add_G0])
            h0 = matrix([h0, add_h0])
        if minNet is not None:
            add_G0 = matrix([ [zn.T], [-1.0*en], [znn]])
            add_h0 = matrix( -float(minNet), (n,1))
            G0 = matrix([G0, add_G0])
            h0 = matrix([h0, add_h0])
            
        solution = socp(coeff, G0, h0, [G1], [h1])
    else:
        #без доп. переменных
        G0 = matrix([ [-1.0], [matrix(0.0, (1,n))] ])
        add_G0 = matrix([[0.0], [matrix(1.0, (1,n))] ])
        G0 = matrix([G0,add_G0])
        add_G0 = matrix([ [zn.T], [-1.0*en]])
        G0 = matrix([G0,add_G0])
        
        h0 = matrix([0.0, maxGross])
        add_h0 = matrix( 0.0, (n,1))
        h0 = matrix([h0, add_h0])
        
        A = matrix([[0.0], [matrix(1.0, (1,n))]])
        b = matrix(float(maxGross), (1,1))
        
        solution = socp(coeff, G0, h0, [G1], [h1], A, b)
        
    if solution['status']=='optimal':
        pi = np.array(solution['x'][1:n+1])
        pi_factors = np.array([]) 
        profit = alpha.dot(pi)
        risk = sqrt( pi.T.dot(cov).dot(pi))
        return pi, pi_factors, risk, profit
    else:
        print("Solution is not found...")
        pi = np.array([0 for i in range(n)])
        pi_factors = np.array([]) 
        profit = 0.
        risk = 0.
        return pi, pi_factors, risk, profit 